# Data Preparation

## PDF-Text extrahieren & vorbereiten

Der Nachhaltigkeitsbericht wird eingelesen, gereinigt und in Abschnitte (Chunks) unterteilt.

In [21]:
import fitz
import re

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

def clean_text(text):
    text = text.replace("\n", " ")
    text = re.sub(r'\s+', ' ', text)  # Mehrfache Leerzeichen entfernen
    return text.strip()

raw_text = extract_text_from_pdf("reports/2023_Volkswagen_Group_Nachhaltigkeitsbericht.pdf")
text = clean_text(raw_text)

## Text in Chunks aufteilen

In [22]:
def chunk_text(text, max_length=500):
    sentences = re.split(r'(?<=[.!?])\s+', text)
    chunks, current_chunk = [], ""
    
    for sentence in sentences:
        if len(current_chunk) + len(sentence) <= max_length:
            current_chunk += " " + sentence
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sentence
    if current_chunk:
        chunks.append(current_chunk.strip())
    
    return chunks

chunks = chunk_text(text)
print(f"Anzahl Chunks: {len(chunks)}")

Anzahl Chunks: 1174


In [23]:
chunks[1]

'2023 N A C H H A LT I G K E I T S B E R I C H T Inhalt Navigationshilfe Kapitelverweise innerhalb des Nachhaltigkeitsberichts Verweise auf Websites 6 Vorwort 9 Über den Bericht 12 \x07 Nachhaltigkeit in der Unternehmens-DNA 13 Corporate Governance 17 Konzernstrategie NEW AUTO 22 \x07ESG-Performance-Management und Wesentlichkeitsanalyse 25 Nachhaltigkeitsmanagement 27 \x07Nachhaltigkeitswirkungen neuer Mobilitätsangebote 30 Stakeholder-Management 33 Umwelt-Compliance-Management 42 Risikomanagement 45 Corporate Citizenship Fokusthemen 1 \x07Dekarbonisierung 48 Dekarbonisierung 69 EU-Taxonomie 2 \x07Kreislaufwirtschaft 83 Kreislaufwirtschaft 3 \x07Menschen in der Transformation 90 Menschen in der Transformation 4 \x07Vielfalt 104 Vielfalt 5 \x07Integrität und Compliance 109 Integrität und Compliance 6 \x07Lieferkette und Menschenrechte 117 Lieferkette und Menschenrechte 131 Anhang 132 \x07Vermerk des unabhängigen Wirtschaftsprüfers 135 Impressum 2 Nachhaltigkeits\xad kommunikation des Vo

## Textverarbeitung

Tokenisierung, Stoppwörter & Lemmatisierung

In [28]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('german'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zäöüß ]', ' ', text)
    tokens = text.split()
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words and len(token) > 2]
    return " ".join(tokens)

preprocessed_chunks = [preprocess_text(chunk) for chunk in chunks]
print(f"Beispiel vorverarbeiteter Chunk:\n{preprocessed_chunks[1][:100]}...\n")

Beispiel vorverarbeiteter Chunk:
inhalt navigationshilfe kapitelverweise innerhalb nachhaltigkeitsberichts verweise website vorwort b...



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\schmu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\schmu\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Filterung von Nachhaltigkeitsversprechen

In diesem Schritt werden die Textabschnitte (Chunks) auf das Vorkommen vordefinierter Schlüsselbegriffe zu Nachhaltigkeit (z. B. klimaneutral, emissionen, paris) überprüft. Nur die Textpassagen, die mindestens einen dieser Begriffe enthalten, werden als potenzielle Nachhaltigkeitsversprechen markiert und für die weitere Analyse berücksichtigt.

In [32]:
relevant_keywords = [ 
    "verantwortung", "nachhaltigkeit", "dekarbonisierung", "strategie",
    "2030", "klimaschutz", "emission",  "deklarieren", "klimaziel", 
    "co2", "paris", "klimaneutralität", "umweltziel"
]

# Keywords normalisiert (klein & lemmatisiert)
lemmatized_keywords = [lemmatizer.lemmatize(w.lower()) for w in relevant_keywords]

def contains_keywords(text):
    return any(keyword in text for keyword in lemmatized_keywords)

print("Keywords:", lemmatized_keywords)
print("Original Chunk Beispiel:\n", chunks[1][:300])
print("Vorverarbeiteter Chunk Beispiel:\n", preprocessed_chunks[1][:300])

# 2. Check Filterfunktion:
print("Enthält Keywords?", contains_keywords(preprocessed_chunks[1]))

# 3. Filterung mit originalen und vorverarbeiteten Chunks verknüpfen
filtered_chunks = [
    original for original, preprocessed in zip(chunks, preprocessed_chunks)
    if contains_keywords(preprocessed)
]

print(f"Gefilterte Chunks: {len(filtered_chunks)}")
print("Beispiel gefilterter Chunk:", filtered_chunks[1][:300])

Keywords: ['verantwortung', 'nachhaltigkeit', 'dekarbonisierung', 'strategie', '2030', 'klimaschutz', 'emission', 'deklarieren', 'klimaziel', 'co2', 'paris', 'klimaneutralität', 'umweltziel']
Original Chunk Beispiel:
 2023 N A C H H A LT I G K E I T S B E R I C H T Inhalt Navigationshilfe Kapitelverweise innerhalb des Nachhaltigkeitsberichts Verweise auf Websites 6 Vorwort 9 Über den Bericht 12  Nachhaltigkeit in der Unternehmens-DNA 13 Corporate Governance 17 Konzernstrategie NEW AUTO 22 ESG-Performance-Manage
Vorverarbeiteter Chunk Beispiel:
 inhalt navigationshilfe kapitelverweise innerhalb nachhaltigkeitsberichts verweise website vorwort bericht nachhaltigkeit unternehmens dna corporate governance konzernstrategie new auto esg performance management wesentlichkeitsanalyse nachhaltigkeitsmanagement nachhaltigkeitswirkungen neuer mobilit
Enthält Keywords? True
Gefilterte Chunks: 468
Beispiel gefilterter Chunk: Auf unserem Weg vom Automobilhersteller zum Mobilitätskonzern ordnen wir

## KeyBERT zur Extraktion der relevantesten Aussagen

In diesem Schritt wird mithilfe von KeyBERT und einem vortrainierten SentenceTransformer-Modell (MiniLM) aus jedem Textabschnitt (chunk) eine Liste der wichtigsten Schlüsselbegriffe (Keywords) extrahiert. Diese Keywords dienen als Grundlage für die semantische Analyse und spätere Filterung von relevanten Nachhaltigkeitsaussagen.

In [38]:
from keybert import KeyBERT
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("paraphrase-MiniLM-L6-v2")
kw_model = KeyBERT(model=model)

german_stopwords = stopwords.words('german')

for i, chunk in enumerate(filtered_chunks[:5]):
    keywords = kw_model.extract_keywords(chunk, keyphrase_ngram_range=(1, 3), stop_words=german_stopwords, top_n=5)
    print(f"Chunk {i+1} Keywords: {keywords}\n")

Chunk 1 Keywords: [('nachhaltigkeitsmanagement 27 nachhaltigkeitswirkungen', 0.6765), ('nachhaltigkeitsmanagement 27', 0.6708), ('nachhaltigkeitsmanagement', 0.6663), ('25 nachhaltigkeitsmanagement', 0.6639), ('wesentlichkeitsanalyse 25 nachhaltigkeitsmanagement', 0.6635)]

Chunk 2 Keywords: [('automobilhersteller mobilitätskonzern ordnen', 0.5576), ('effizienter innovativer', 0.5324), ('automobilhersteller mobilitätskonzern', 0.5283), ('effizienter innovativer kundennäher', 0.525), ('zukunftsgerichtet dabei behalten', 0.5071)]

Chunk 3 Keywords: [('definiert bereich nachhaltigkeit', 0.6284), ('vorliegende nachhaltigkeitsbericht', 0.6078), ('fokusthemen vorliegende nachhaltigkeitsbericht', 0.6011), ('bereich nachhaltigkeit esg', 0.5966), ('nachhaltigkeit esg konzerninitiative', 0.5831)]

Chunk 4 Keywords: [('nachhaltigkeit unternehmens dna', 0.6578), ('nachhaltigkeit unternehmen', 0.6219), ('dna definition nachhaltigkeit', 0.6189), ('nachhaltigkeit unternehmen einblick', 0.6164), ('nac

In [43]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer

preprocessed_filtered_chunks = [preprocess_text(chunk) for chunk in filtered_chunks]

embedding_model = SentenceTransformer("paraphrase-MiniLM-L6-v2")
topic_model = BERTopic(language="german", embedding_model=embedding_model)

topics, probs = topic_model.fit_transform(preprocessed_filtered_chunks)

topic_info = topic_model.get_topic_info()
print(topic_info)

print("\nBeispiel Topic 1:")
print(topic_model.get_topic(1))

   Topic  Count                                       Name  \
0      0    334      0_volkswagen_konzern_emissionen_sowie   
1      1    134  1_compliance_integrität_menschen_vielfalt   

                                      Representation  \
0  [volkswagen, konzern, emissionen, sowie, konze...   
1  [compliance, integrität, menschen, vielfalt, m...   

                                 Representative_Docs  
0  [bestandteil zusammengefassten nichtfinanziell...  
1  [inhalt unternehmens dna dekarbonisierung krei...  

Beispiel Topic 1:
[('compliance', 0.11401316522868743), ('integrität', 0.09951560714829692), ('menschen', 0.09878921585524367), ('vielfalt', 0.09822713177513791), ('menschenrechte', 0.09789965164703741), ('transformation', 0.09678795553085918), ('dekarbonisierung', 0.09540355140999837), ('lieferkette', 0.09514045177367722), ('kreislaufwirtschaft', 0.09417880239280832), ('dna', 0.09411667294955907)]


## Artikel Vorverarbeitung

In [56]:
from newspaper import Article

url = "https://www.arbeit-wirtschaft.at/greenwashing-tarnen-und-taeuschen/"
article = Article(url, language='de')
article.download()
article.parse()
article_text = article.text

print(text)

Unternehmen verursachen einen Großteil der globalen CO2-Emissionen. Nach eigenen Angaben sind sie bemüht, den Ausstoß zu senken. Doch meist steckt nur Greenwashing dahinter.

Der Wind hat sich gedreht. Ab dem Jahr 2004 diskutierte die Welt zunächst über den CO2-Fußabdruck, den jeder Mensch hinterlässt. Der Energiekonzern BP hatte einen Online-Rechner eingeführt. Jede:r konnte sich selbst seine:ihre Portion Schuld an der Klimakatastrophe ausrechnen. Mittlerweile hat sich aber die Erkenntnis durchgesetzt, dass Unternehmen rund 71 Prozent der globalen CO2-Emissionen verursachen. Die Energieindustrie alleine bläst fast doppelt so viel Emissionen in die Luft wie der Transportsektor. Und versucht das, durch Greenwashing zu verschleiern.

Denn Konzerne sind verpflichtet, einen Nachhaltigkeitsbericht zu veröffentlichen. Darin fassen die Firmen ihre Bemühungen zur Reduktion ihres CO2-Fußabdrucks zusammen. Kontrolliert werden diese Ziele unter anderem vom Corporate Climate Responsibility Monitor

In [57]:
cleaned_article_text = clean_text(article_text)
article_chunks = chunk_text(cleaned_article_text)
print(f"Anzahl Artikel-Chunks: {len(article_chunks)}")

Anzahl Artikel-Chunks: 8


In [58]:
preprocessed_article_chunks = [preprocess_text(chunk) for chunk in article_chunks]

print(f"Beispiel vorverarbeiteter Chunk:\n{preprocessed_article_chunks[0][:200]}")

Beispiel vorverarbeiteter Chunk:
unternehmen verursachen großteil globalen emissionen eigenen angaben bemüht ausstoß senken meist steckt greenwashing dahinter wind gedreht jahr diskutierte welt zunächst fußabdruck mensch hinterlässt 


In [ ]:
#für kleine Datenmengen
from umap import UMAP
import hdbscan

embedding_model = SentenceTransformer("paraphrase-MiniLM-L6-v2")

umap_model = UMAP(n_neighbors=3, n_components=5, metric='cosine')
hdbscan_model = hdbscan.HDBSCAN(min_cluster_size=2, metric='euclidean', prediction_data=True)

topic_model = BERTopic(
    language="german",
    embedding_model=embedding_model,
    umap_model=umap_model,
    hdbscan_model=hdbscan_model
)

topics, probs = topic_model.fit_transform(preprocessed_article_chunks)

topic_info = topic_model.get_topic_info()
print(topic_info)

print("\nBeispiel Topic 1:")
print(topic_model.get_topic(1))

   Topic  Count                                            Name  \
0     -1      4                   -1_unternehmen_ccrm_sei_seien   
1      0      2  0_emissionen_verursachen_greenwashing_globalen   
2      1      2              1_prozent_firmen_emissionen_maersk   

                                      Representation  \
0  [unternehmen, ccrm, sei, seien, autobauer, ger...   
1  [emissionen, verursachen, greenwashing, global...   
2  [prozent, firmen, emissionen, maersk, apple, g...   

                                 Representative_Docs  
0  [kontrolliert ziele corporate climate responsi...  
1  [mittlerweile erkenntnis durchgesetzt unterneh...  
2  [festgestellt meisten strategien unternehmen b...  

Beispiel Topic 1:
[('prozent', 0.1292642503588437), ('firmen', 0.12043215657900685), ('emissionen', 0.11329583488703311), ('maersk', 0.10479891494707327), ('apple', 0.10479891494707327), ('group', 0.10479891494707327), ('gerade', 0.10479891494707327), ('überhaupt', 0.10479891494707327

# Modeling

## Vergleich von Artikel und Bericht

In [64]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

all_report_embeddings = embedding_model.encode(preprocessed_filtered_chunks, convert_to_tensor=True)
all_article_embeddings = embedding_model.encode(preprocessed_article_chunks, convert_to_tensor=True)

similarity_matrix = cosine_similarity(all_article_embeddings, all_report_embeddings)

top_k = 3

for i, article_chunk in enumerate(preprocessed_article_chunks):
    similarities = similarity_matrix[i]
    top_indices = np.argsort(similarities)[::-1][:top_k]
    
    print(f"\nArtikel-Chunk {i+1}:\n{article_chunks[i][:300]}...\n")
    
    for rank, idx in enumerate(top_indices):
        print(f"Ähnlichster Bericht-Chunk {rank+1} (Score: {similarities[idx]:.2f}):")
        print(filtered_chunks[idx][:300])
        print("-" * 80)


Artikel-Chunk 1:
Unternehmen verursachen einen Großteil der globalen CO2-Emissionen. Nach eigenen Angaben sind sie bemüht, den Ausstoß zu senken. Doch meist steckt nur Greenwashing dahinter. Der Wind hat sich gedreht. Ab dem Jahr 2004 diskutierte die Welt zunächst über den CO2-Fußabdruck, den jeder Mensch hinterläss...

Ähnlichster Bericht-Chunk 1 (Score: 0.81):
Klimaschutzbezogene Begrifflichkeiten Die Verwendung des Begriffs CO2-Emissionen in diesem Be­ richt schließt die Betrachtung und Ermittlung weiterer klima­ schädlicher Treibhausgase wie zum Beispiel Methan (CH₄) und Lachgas (N₂O) mit ein (CO2-Äquivalente). Alle Angaben in diesem Bericht zu CO2-Emis
--------------------------------------------------------------------------------
Ähnlichster Bericht-Chunk 2 (Score: 0.81):
Es kann unterschiedliche Gründe geben, die Emissionen vergangener Jahre neu zu ­bewerten, um einen fairen Vergleich mit aktuellen Emissionen zu ermöglichen: Strukturelle Änderungen des Unterneh­ mens, Änderung